### 全局基础

In [1]:
# 导入包
import matplotlib.pyplot as plt
import numpy as np
import scipy
import multiprocessing as mp

### 方程求解

$z''(t)=\frac{H_0^4\kappa C_1\Omega_{1,0}^2(z(t)^4+1)+3H_0^4\Omega_{1,0}^2z(t)^2(2\kappa C_1-3z'(t))+H_0^4\Omega_{1,0}^2z(t)^3(4\kappa C_1-3z'(t))-3H_0^4\Omega_{1,0}^2z'(t)+5H_0^2\Omega_{1,0}z'(t)^3-\kappa C_1z'(t)^4+H_0^2\Omega_{1,0}z(t)(4H_0^2\kappa C_1\Omega_{1,0}-9H_0^2\Omega_{1,0}z'(t)+5z'(t)^3)}{2H_0^2\Omega_{1,0}(1+z(t))^2z'(t)}$

In [2]:
import solution

### OHD(only CC)

$H(z)=-\dfrac{1}{1+z}\dfrac{\text{d}z}{\text{d}t}$

In [ ]:
from OHD.result import main

if __name__ == '__main__':
    mp.freeze_support()
    main()

### SNe Ia

$\displaystyle d_L=c(1+z)\int_0^z\frac{dz'}{H(z')}=c(1+z)\left[t_0-t+\int_t^{t_0}z(t')dt'\right]$

$m=M+5\log(d_L)+25$

In [ ]:
# panthon
from SNe.result1 import main

if __name__ == '__main__':
    mp.freeze_support()
    main()

In [ ]:
# panthon+
from SNe.result2 import main

if __name__ == '__main__':
    mp.freeze_support()
    main()

### BAO

$\displaystyle D_M(z)=\frac{c}{H_0\sqrt{\Omega_K}}\sinh\left[\sqrt{\Omega_K}\int_0^z\frac{\text{d}z'}{H(z')/H_0}\right]=r_d/\Delta\theta\\$  
when $\displaystyle\Omega_k\to0,D_M(z)=c\int_0^z\frac{\text{d}z'}{H(z')}$

$D_H(z)=c/H(z)=r_d/\Delta z$

$F_{AP}(z)=D_M(z)/D_{H}(z)$

$\displaystyle D_V(z)=(zD_M(z)^2D_H(z))^{1/3}=c\left[z\left(\int_0^z\frac{\text{d}z'}{H(z')}\right)^2\frac{1}{H(z)}\right]^{1/3}$

$\displaystyle r_d=\int_{z_d}^{\infty}\frac{c_s(z)}{H(z)}\text{d}z$

$\displaystyle c_s(z)=\frac{c}{\sqrt{3\left(1+\dfrac{3\rho_B(z)}{4\rho_{\gamma}(z)}\right)}}$

In [ ]:
from BAO.result import main

if __name__ == '__main__':
    mp.freeze_support()
    main()

### CMB